## Тестовое (data engineering)
### Шкуратова Д.В.

## 1. Схема БД

### Users
| Поле       | Тип данных   | Ограничения      | Описание                      |
|------------|--------------|------------------|-------------------------------|
| guid       | uuid         | primary key      |                               |
| username   | varchar(50)  | unique, not null |                               |
| email      | varchar(255) | unique, not null |                               |
| date_joined | timestamp     | not null         | Дата регистрации пользователя |

### Topics 
| Поле         | Тип данных   | Ограничения           | Описание                |
|--------------|--------------|-----------------------|-------------------------|
| guid         | uuid         | primary key           |                         |
| user_guid    | uuid         | foreign key, not null | Ссылка на таблицу Users |
| title        | varchar(500) | not null              | Заголовок темы форума   |
| content      | text         |                       | Содержание темы         |
| date_created | timestamp    | not null              | Дата создания темы      |

### Messages 
| Поле         | Тип данных | Ограничения           | Описание                                                                            |
|--------------|------------|-----------------------|-------------------------------------------------------------------------------------|
| guid         | uuid       | primary key           |                                                                                     |
| theme_guid   | uuid       | foreign key, not null | Ссылка на таблицу Topics, при удалении темы происходит каскадное удаление сообщений |
| user_guid    | uuid       | foreign key           | Ссылка на таблицу Users, может принимать значение null для отслеживания анонимов    |
| body         | text       | not null              | Текст сообщения                                                                     |
| date_created | timestamp  | not null              | Дата написания сообщения                                                            |
        
### Log_event
| Поле       | Тип данных | Ограничения | Описание                                       |
|------------|------------|-------------|------------------------------------------------|
| uuid       | uuid       | primary key |                                                |
| event_name | varchar    | not null    | Виды действий пользователей |
## Logs

 | Поле                 | Тип данных    | Ограничения | Описание                                                                       |
|----------------------|---------------|-------------|--------------------------------------------------------------------------------|
| guid                 | uuid          | primary key |                                                                                |
| user_guid            | uuid          | foreign key | Ссылка на таблицу Users, при удалении строки из Users заменяется на null       |
| topic_guid           | uuid          | foreign key | Ссылка на таблицу Topics, при удалении строки из Topics заменяется на null     |
| message_guid         | uuid          | foreign key | Ссылка на таблицу Messages, при удалении строки из Messages заменяется на null |
| event_guid           | uuid          | foreign key | Ссылка на таблицу log_events                                                   | 
| log_date             | timestamp     | not null    | Дата действия                                                                  |
| response_status_code | int           | not null    | Oтвет сервера: 1 - успех, 0 - ошибка                                           | 
| log_message          | varchar |             | Описание действия                                                              | 


В таблице Logs хранятся логи для всех сущностей БД, поэтому каждый из внешниех ключей (user_guid, topic_guid, messages_guid) может принимать значение null
  

![logsddl.png](logsddl.png)

## Партицирование таблицы Logs

Если в таблице Logs будет храниться очень большое количество записей для каждого дня, а составлять отчеты нужно, например, только за последние месяцы, то можно настроить секционирование таблицы. Тогда для каждого месяца нужно создавать отдельную таблицу.


Пример секционирования таблицы Logs, с созданной секцией, в которой будут храниться только записи за февраль 2025 года. Секции могут унаследовать только ограничение not null, первичные и внешние ключи, ограничения исключения нужно задавать для каждой секции отдельно. 
```PostgreSQL
create table logs(
    guid uuid,
    log_date timestamp not null,
    user_guid uuid,
    topic_guid uuid,
    message_guid uuid,
    response_status_code int not null ,
    log_message varchar(1024),
    event_guid uuid

) partition by range (log_date);

create table logs_y2025m02 partition of logs
for values from ('2025-02-01') to ('2025-03-01');

alter table logs_y2025m02 add constraint logs_y2025m02_pk primary key (guid);
alter table logs_y2025m02 add constraint user_logs_fk
    foreign key (user_guid) references users (guid) on delete set null;
alter table logs_y2025m02 add constraint topic_logs_fk
    foreign key (topic_guid) references topics (guid) on delete set null;
alter table logs_y2025m02 add constraint message_logs_fk
    foreign key (message_guid) references messages (guid) on delete set null;
alter table logs_y2025m02 add constraint event_logs_fk
    foreign key (event_guid) references log_event (guid);
alter table logs_y2025m02 add constraint valid_status_code check ( response_status_code in (0,1) );
```


## 2. Генерация данных

Generator - принимает на вход дату начала и дату окончания.
 
Методы:
- **gen_data** 
    - Генерирует данные для каждого типа логов на каждый день временного отрезка
- **registration** 
    - Генерирует создание аккаунтов пользователей, для некоторых действий возвращается ошибка 
- **first_visited** 
    - Генерирует первое посещение на сайт
- **login** 
    - Генерирует авторизацию пользователей, для выбора user_guid генерируется выборка из уже созданных пользователей, для некоторых возвращается ошибка по причине неверного логина/пароля
- **logout** 
    - Генерирует логаут пользователей
- **create_topic** 
    - Создание тем, для некоторых возвращается ошибка по причине отсутствия логина
- **entry_topic** 
    - Генерирует посещение темы пользователем, случайно выбирается количество авторизированных и анонимных пользователей и для них генерируется выборка из существующих тем форума
- **delete-topic** 
    - Генерирует удаление тем, при удалении темы значение topic_guid, message_guid, для сообщений, привязанных к удаленной теме заменяется на null
- **write_message** 
    - Генерирует сообщения для выборки с авторизированными и анонимными пользователями


Для каждого действия установлен минимальный порог - 5 записей, для ошибок - 2

**script.py** - скрипт заполнения БД сгенерированными данными с помощью библиотеки psycopg2



# 3. Docker

#### Сервисы:
1. **database** - контейнер с БД, расширенный скриптом с ddl запросами
2. **seed_db** - контейнер со скриптом генерации датасета

## 4. Скрипт агрегации данных

На вход **aggregate.py** передается начальная и конечная дата периода расчета в формате YYYY-DD-MM.

Результат запроса сохраняется в csv файл, который содержит:
- log_date - дата, округленная до дня
- new_users - количество новых пользователей за день
- anon_pcnt - процент сообщений, написанных анонимами за день
- msg_cnt - общее количество сообщений за день
- new_topic_pcnt - процент изменения количества тем на форуме относительно предыдущего дня



In [1]:
from sqlalchemy import create_engine
import pandas as pd

host = 'localhost:5432'
user = 'admin'
database_name = 'logsdb'
password = 'admin'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database_name}')

In [3]:
with open('../Task4/request.sql') as f:
    query = f.read()

pd.read_sql(query, engine)

,log_date,new_users,anon_pcnt,msg_cnt,new_topic_pcnt
0,2025-02-10,20,51,170,0.0
1,2025-02-11,20,50,238,400.0
2,2025-02-12,7,50,361,300.0
3,2025-02-13,5,47,432,30.0
4,2025-02-14,5,44,290,0.0
5,2025-02-15,19,39,46,211.5
6,2025-02-16,8,48,80,28.4
7,2025-02-17,2,46,412,10.6
8,2025-02-18,24,47,297,9.6
9,2025-02-19,16,48,496,-0.8
